# ODSs and HFCs


From Fiona's paper (https://acp.copernicus.org/articles/21/1211/2021/), ODS forcing is -0.18, and we know the ozone contribution is -0.33, so the direct forcing contribution from ODSs is +0.15.

**There is an ACI effect from Ozone**

The total GHG forcing is +2.93, and the sum of CO2, CH4, N2O and ODSs is +2.92. Things might be non-linear and internal variability might be affecting things, but this means that HFCs = +0.01.

In [ ]:
from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
import pooch
from scipy.interpolate import interp1d
from scipy.optimize import root

In [ ]:
climate_response_df = pd.read_csv('../data/fair-calibrations/4xCO2_energy_balance_ebm3.csv')

In [ ]:
rcmip_emissions_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-emissions-annual-means-v5-1-0.csv",
    known_hash="md5:4044106f55ca65b094670e7577eaf9b3",
)
emis_df = pd.read_csv(rcmip_emissions_file)

In [ ]:
rcmip_concentration_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-concentrations-annual-means-v5-1-0.csv",
    known_hash="md5:0d82c3c3cdd4dd632b2bb9449a5c315f",
)
conc_df = pd.read_csv(rcmip_concentration_file)

In [ ]:
ods = ['CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115', 'HCFC-22', 'HCFC-141b', 'HCFC-142b',
        'CCl4', 'CHCl3', 'CH2Cl2', 'CH3Cl', 'CH3CCl3', 'CH3Br', 'Halon-1211', 'Halon-1301', 'Halon-2402']

hfc = ['CF4', 'C2F6', 'C3F8', 'c-C4F8', 'C4F10', 'C5F12', 'C6F14', 'C7F16', 'C8F18', 'NF3', 'SF6', 'SO2F2',
        'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32',
        'HFC-365mfc', 'HFC-4310mee']

ghgs = ['CO2', 'CH4', 'N2O'] + ods + hfc

In [ ]:
species_to_rcmip = {specie: specie.replace("-", "") for specie in ghgs}

In [ ]:
# iterative tuning
target_hfc = +0.01
target_ods = +0.15

In [ ]:
def erf_rootfinder(x):
    ods_scale, hfc_scale = x
    f = FAIR(temperature_prescribed=True)

    f.define_time(1850, 2015, 1)
    scenarios = ['ssp245']
    f.define_scenarios(scenarios)
    configs = ['UKESM1-0-LL']
    f.define_configs(configs)
    species, properties = read_properties(filename='../data/species_configs_properties_vanilla.csv')
    species.remove('CO2 FFI')      # c-driven run
    species.remove('CO2 AFOLU')    # c-driven run
    species.remove('Halon-1202')   # not in CMIP6 list of species
    species.remove('Contrails')    # not modelled in UKESM, I think
    species.remove('NOx aviation') # which renders this redundant
    species.remove('Light absorbing particles on snow and ice')  # I believe not modelled in UKESM
    species.remove('Land use')     # Nothing here affects or is affected by when we only want GHG forcing
    species.remove('Solar')        # Not needed
    species.remove('Volcanic')     # Not needed
    del properties['CO2 FFI']
    del properties['CO2 AFOLU']
    del properties['Halon-1202']
    del properties['Contrails']
    del properties['NOx aviation']
    del properties['Light absorbing particles on snow and ice']
    del properties['Land use']
    del properties['Solar']
    del properties['Volcanic']

    for specie in ghgs:
        properties[specie]['input_mode'] = 'concentration'
    f.define_species(species, properties)
    f.allocate()
    
    fill(f.climate_configs['ocean_heat_capacity'], climate_response_df.loc[0, 'C1':'C3'])
    fill(f.climate_configs['ocean_heat_transfer'], climate_response_df.loc[0, 'kappa1':'kappa3'])
    fill(f.climate_configs['deep_ocean_efficacy'], climate_response_df.loc[0, 'epsilon'])
    fill(f.climate_configs['gamma_autocorrelation'], climate_response_df.loc[0, 'gamma'])
    
    initialise(f.forcing, 0)
    fill(f.temperature, 0)
    
    for specie in ghgs:

        conc = conc_df.loc[(conc_df['Variable'].str.endswith('|' + species_to_rcmip[specie])) & 
                        (conc_df['Scenario']=='historical') &
                        (conc_df['Region']=='World'), '1850':'2015'].values.squeeze()
        conc[-1] = conc[-2]
        if specie in ods + hfc:
            fill(
                f.concentration.loc[
                    dict(specie=specie, timebounds=slice(1850,2016), scenario='ssp245', config='UKESM1-0-LL')],
                conc
            )       
        else:
            fill(
                f.concentration.loc[
                    dict(specie=specie, timebounds=slice(1850,2016), scenario='ssp245', config='UKESM1-0-LL')],
                conc[0]
            )

    f.fill_species_configs(filename='../data/species_configs_properties_vanilla.csv')
    f.species_configs['baseline_concentration'].loc[dict(specie=specie)] = conc[0]
    
    
    # fill emissions and baseline emissions with 1850 values
    for specie in ['Sulfur', 'BC', 'OC', 'VOC', 'NOx', 'CO', 'NH3']:
        emis = emis_df.loc[(emis_df['Variable'].str.endswith('|' + specie)) & 
                    (emis_df['Scenario']=='ssp245') &
                    (emis_df['Region']=='World'), '1850'].values[0]

        f.emissions.loc[dict(specie=specie, timepoints=slice(1850.5,2015), scenario='ssp245', config='UKESM1-0-LL')] = emis
        f.species_configs['baseline_emissions'].loc[dict(specie=specie)] = emis
        
    # fill concentrations and baseline concentrations
    for specie in ghgs:

        conc = conc_df.loc[(conc_df['Variable'].str.endswith('|' + species_to_rcmip[specie])) & 
                        (conc_df['Scenario']=='historical') &
                        (conc_df['Region']=='World'), '1850':'2015'].values.squeeze()
        conc[-1] = conc[-2]
        if specie in ods + hfc:
            fill(
                f.concentration.loc[
                    dict(specie=specie, timebounds=slice(1850,2016), scenario='ssp245', config='UKESM1-0-LL')],
                conc
            )

        else:
            fill(
                f.concentration.loc[
                    dict(specie=specie, timebounds=slice(1850,2016), scenario='ssp245', config='UKESM1-0-LL')],
                conc[0]
            )
        f.species_configs['baseline_concentration'].loc[dict(specie=specie)] = conc[0]

    f.species_configs['baseline_concentration'].loc[dict(specie='Equivalent effective stratospheric chlorine')] = 344.36275911
    
    # pre-calibrated for methane, aerosol, CO2, N2O and ozone runs: do not adjust
    f.species_configs['forcing_scale'].loc[dict(specie="CH4")] = 1.11547955
    f.species_configs['h2o_stratospheric_factor'].loc[dict(specie="CH4")] = 1.29652705e-01
    f.species_configs['ozone_radiative_efficiency'].loc[dict(specie="CH4")] = 1.27049657e-04
    f.species_configs['ozone_radiative_efficiency'].loc[dict(specie="Equivalent effective stratospheric chlorine")] = -0.00029119797470220245
    f.species_configs['ozone_radiative_efficiency'].loc[dict(specie="N2O")] = 0.0007481397748679878
    f.species_configs['ozone_radiative_efficiency'].loc[dict(specie="VOC")] = 0.0006596999582126578
    f.species_configs['ozone_radiative_efficiency'].loc[dict(specie="NOx")] = 0.000984642055228785
    f.species_configs['forcing_temperature_feedback'].loc[dict(specie="Ozone")] = -0.09693932216231645  # (13)
    f.species_configs['forcing_scale'].loc[dict(specie="N2O")] = 1.16044956348451
    f.species_configs['forcing_scale'].loc[dict(specie="CO2")] = 1.04321901751203

    for specie in species:
        f.species_configs['erfari_radiative_efficiency'].loc[dict(specie=specie)] = 0
    f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='Sulfur')] = -0.00283793
    f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='BC')] = 0.01757433
    f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='OC')] = -0.0028512
    f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='CH4')] = -5.09377882e-05
    f.species_configs['aci_scale'].loc[dict(config='UKESM1-0-LL')] = -8.22336281e-01
    f.species_configs['aci_shape'].loc[dict(specie='Sulfur')] = 3.01888722e-02
    f.species_configs['aci_shape'].loc[dict(specie='BC')] = 5.07111186e-30
    f.species_configs['aci_shape'].loc[dict(specie='OC')] = 6.60381587e-16
    f.species_configs['aci_shape'].loc[dict(specie="CH4")] = -2.61186114e-04

    # This is what we are adjusting
    for specie in ods:
        f.species_configs['forcing_scale'].loc[dict(specie=specie)] = ods_scale
    for specie in hfc:
        f.species_configs['forcing_scale'].loc[dict(specie=specie)] = hfc_scale
    
    f.run(progress=False)
    f_hfc = f.forcing[-1,0,0,27:50].sum().data
    f_ods = f.forcing_sum[-1,0,0].data - f_hfc
    print(f_hfc, f_ods)
    return np.array(
        [
            f_ods - target_ods, 
            f_hfc - target_hfc, 
        ]
    )

In [ ]:
solution = root(
    erf_rootfinder, 
    np.array(
        [
            1,
            1,
        ],
    ),
    method='lm',
    options={'maxiter': 500000}
)

In [ ]:
solution.x